In [ ]:
import os

#os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from torch import nn
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, DataCollatorWithPadding

from transformers import DataCollatorForTokenClassification
import torch
from pathlib import Path
import re
from datasets import load_dataset

from sklearn.model_selection import train_test_split

import time
import wandb

from matplotlib import pyplot as plt

In [ ]:
data=pd.read_csv('data.csv')
data.describe()

In [ ]:
plt.hist(data['label'])
plt.show()

In [ ]:
data=pd.read_csv('data.csv')
classes=np.sort(data['label'].drop_duplicates().values)
classes_dict={}
for i,cls in enumerate(classes):
    classes_dict[str(cls)]=i
print(len(classes_dict))
data['label']=data['label'].map(lambda x: classes_dict[str(x)])

max_len=[len(line) for line in data['text']]
print(max(max_len))
data.head(5)

In [ ]:
id2label = {str(i):label for i, label in enumerate(classes)}
label2id = {label:str(i) for i, label in enumerate(classes)}

In [ ]:
id2label

In [ ]:
label2id

In [ ]:
dataset = Dataset.from_pandas(data,preserve_index=False)
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
model_name='sberbank-ai/ruBert-base'
# model_name='distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=9)
model.config.id2label = id2label
model.config.label2id = label2id
model.config._num_labels = len(id2label)  ## replacing 9 by 13
model.config.num_labels = len(id2label)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[p for (p, l) in zip(prediction, label) if l != -100] for prediction, label in
                        zip(predictions, labels)]
    true_labels = [[l for (p, l) in zip(prediction, label) if l != -100] for prediction, label in
                   zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    print(results)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"]}

In [ ]:
# metric = load_metric("accuracy")
metric = load_metric("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # if task != "stsb":
    predictions = np.argmax(predictions, axis=1)
    # else:
    # predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
batch_size=16
epochs=4

args = TrainingArguments(
    f"test-classify-{str(time.time())}",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    learning_rate=1e-3,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    logging_strategy='steps',
    logging_first_step=True,
    logging_steps=5,
    report_to='wandb',
    # fp16=True,
    weight_decay=1e-5,
    dataloader_num_workers=4,
    metric_for_best_model='accuracy'
)

# args = TrainingArguments(
#     f"test-classify-{str(time.time())}",
#     evaluation_strategy="epoch",
#     learning_rate=1e-3,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=epochs,
#     logging_steps=10,
#     weight_decay=1e-2,
#     report_to='wandb'
# )





data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length')

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
wandb.finish()

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets['test'])

In [ ]:
model.save_pretrained("mazur")
tokenizer.save_pretrained("mazur")

In [ ]:
# load the model/tokenizer
from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [ ]:

trainer = Trainer(model=model)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
def pipeline_prediction(text):
    df=pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df,preserve_index=False)
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets)
    return(raw_pred[0][0])